In [1]:
# pip install pyspark

In [2]:
# pip install findspark

In [3]:
import findspark

In [4]:
findspark.init()

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.types import DoubleType,IntegerType
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Lab assignment") \
    .getOrCreate()

matches_df  = spark.read.csv(r"C:\Users\jagad\Desktop\WorldCupMatches.csv", header=True)


In [6]:
matches_df.show()

+----+--------------------+-----------------+--------------------+-----------+--------------+---------------+---------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|Year|            Datetime|            Stage|             Stadium|       City|Home Team Name|Home Team Goals|Away Team Goals|Away Team Name|      Win conditions|Attendance|Half-time Home Goals|Half-time Away Goals|             Referee|         Assistant 1|         Assistant 2|RoundID|MatchID|Home Team Initials|Away Team Initials|
+----+--------------------+-----------------+--------------------+-----------+--------------+---------------+---------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|193

In [7]:
matches_df = matches_df.withColumnRenamed('city','place')
matches_df = matches_df.withColumnRenamed('Away Team Goals','Opponent_Team_Goals')
matches_df = matches_df.withColumnRenamed('Away Team Name','Opponent_Team_Name')
matches_df = matches_df.withColumnRenamed('Attendance','Spectator_Count')
matches_df = matches_df.withColumnRenamed('Stage','League_Stage')
matches_df = matches_df.withColumnRenamed('Datetime','Date_Time')
matches_df = matches_df.withColumnRenamed('Half-time Away Goals','Half-time_Opponent_Goals')
matches_df = matches_df.withColumnRenamed('Home Team Goals','Home_Team_Goals')
matches_df = matches_df.withColumnRenamed('Home Team Name','Home_Team_Name')
matches_df = matches_df.withColumnRenamed('Win conditions','Win_conditions')
matches_df = matches_df.withColumnRenamed('Half-time Home Goals','Half-time_Home_Goals')
matches_df = matches_df.withColumnRenamed('Assistant 1','Assistant_1')
matches_df = matches_df.withColumnRenamed('Assistant 2','Assistant_2')
matches_df = matches_df.withColumnRenamed('Home Team Initials','Home_Team_Initials')
matches_df = matches_df.withColumnRenamed('Away Team Initials','Away_Team_Initials')

In [8]:
matches_df.printSchema


<bound method DataFrame.printSchema of DataFrame[Year: string, Date_Time: string, League_Stage: string, Stadium: string, place: string, Home_Team_Name: string, Home_Team_Goals: string, Opponent_Team_Goals: string, Opponent_Team_Name: string, Win_conditions: string, Spectator_Count: string, Half-time_Home_Goals: string, Half-time_Opponent_Goals: string, Referee: string, Assistant_1: string, Assistant_2: string, RoundID: string, MatchID: string, Home_Team_Initials: string, Away_Team_Initials: string]>

In [9]:
matches_df  = matches_df.withColumn('Spectator_Count', matches_df['Spectator_Count'].cast(IntegerType()))
matches_df  = matches_df.withColumn('Opponent_Team_Goals', matches_df['Opponent_Team_Goals'].cast(IntegerType()))
matches_df = matches_df.withColumn('Home_Team_Goals', matches_df['Home_Team_Goals'].cast(IntegerType()))
matches_df = matches_df.withColumn('Half-time_Home_Goals', matches_df['Half-time_Home_Goals'].cast(IntegerType()))
matches_df = matches_df.withColumn('Half-time_Opponent_Goals', matches_df['Half-time_Opponent_Goals'].cast(IntegerType()))
matches_df = matches_df.withColumn('Home_Team_Goals', matches_df['Home_Team_Goals'].cast(IntegerType()))

In [10]:
#query 1 top 5 home teams with highest number of goals
matches_df.createOrReplaceTempView("WorldCupMatches")
matches_df.select('Home_Team_Name','Home_Team_Goals').orderBy('Home_Team_Goals',ascending=False).show(5)


+--------------+---------------+
|Home_Team_Name|Home_Team_Goals|
+--------------+---------------+
|       Hungary|             10|
|       Hungary|              9|
|    Yugoslavia|              9|
|        Sweden|              8|
|       Uruguay|              8|
+--------------+---------------+
only showing top 5 rows



In [11]:
#query 2: Top 10 matches with highest number of goals
# spark.sql('select sum(Home_Team_Goals,Away_Team_Goals) from  ').show()
a = matches_df.withColumn("Total_Goals", matches_df["Home_Team_Goals"] + matches_df["Opponent_Team_Goals"] + matches_df["Half-time_Home_Goals"] + matches_df["Half-time_Opponent_Goals"]).select('MatchID','Total_Goals')
a.orderBy("Total_Goals",ascending=False).show(10)

+-------+-----------+
|MatchID|Total_Goals|
+-------+-----------+
|   1237|         21|
|   2186|         15|
|   1277|         15|
|   1386|         14|
|    896|         14|
|   1086|         13|
|   1284|         13|
|   1294|         13|
|   1382|         13|
|   1702|         13|
+-------+-----------+
only showing top 10 rows



In [12]:
#query 3number of matches held year wise
# spark.sql('select year,count(*) as number_of_matches from WorldCupMatches group by year').show()
matches_df.select('year').groupBy('year').count().show()

+----+-----+
|year|count|
+----+-----+
|1930|   18|
|1934|   17|
|1938|   18|
|1950|   22|
|1954|   26|
|1958|   35|
|1962|   32|
|1966|   32|
|1970|   32|
|1974|   38|
|1978|   38|
|1982|   52|
|1986|   52|
|1990|   52|
|1994|   52|
|1998|   64|
|2002|   64|
|2006|   64|
|2010|   64|
|2014|   80|
+----+-----+
only showing top 20 rows



In [13]:
# query 4 number of matches in each group stage
matches_df.select('League_Stage').groupBy('League_Stage').count().show()

+--------------------+-----+
|        League_Stage|count|
+--------------------+-----+
|             Group 1|   62|
|             Group 4|   55|
|             Group 2|   59|
|             Group 3|   56|
|         Semi-finals|   36|
|               Final|   20|
|   Preliminary round|    8|
|      Quarter-finals|   66|
|Match for third p...|   15|
|         First round|    9|
|             Group 6|   12|
|             Group B|   60|
|             Group A|   60|
|             Group 5|    6|
|             Group C|   48|
|             Group D|   48|
|             Group F|   48|
|             Group E|   48|
|         Round of 16|   72|
|             Group H|   30|
+--------------------+-----+
only showing top 20 rows



In [14]:
#query 5 Teams with the count of winning matches
a =spark.sql("select CASE WHEN Home_Team_Goals > Opponent_Team_Goals THEN Home_Team_Name ELSE Opponent_Team_Name end as WonTeam  from WorldCupMatches")
a.groupBy('WonTeam').count().show()

+--------------+-----+
|       WonTeam|count|
+--------------+-----+
|        France|   35|
|           USA|   12|
|    Yugoslavia|   21|
|       Romania|   11|
|     Argentina|   54|
|         Chile|   16|
|       Uruguay|   25|
|        Brazil|   75|
|      Paraguay|   11|
|       Austria|   15|
|       Hungary|   16|
|   Switzerland|   14|
|        Sweden|   22|
|       Germany|   35|
|         Spain|   34|
|         Italy|   52|
|Czechoslovakia|   15|
|          Cuba|    1|
|       England|   35|
|    Germany FR|   37|
+--------------+-----+
only showing top 20 rows



In [15]:
#query 6 matches held with extra time given
b =matches_df.select(matches_df["MatchID"]).filter(matches_df["Win_conditions"].contains("extra time")).show()

+-------+
|MatchID|
+-------+
|   1104|
|   1134|
|   1179|
|   1150|
|   1172|
|   1295|
|   1422|
|   1633|
|   1811|
|   1925|
|   1838|
|   2198|
|    432|
|    422|
|    102|
|    181|
|     55|
|    103|
|   3091|
|   8776|
+-------+
only showing top 20 rows



In [16]:
#query 7 top 5 stadiums occupied highest number of spectators for the matches in dataset
matches_df.select('Stadium','Spectator_Count').groupBy('Stadium').sum().orderBy('sum(Spectator_Count)',ascending=False).show(5)

+--------------------+--------------------+
|             Stadium|sum(Spectator_Count)|
+--------------------+--------------------+
|      Estadio Azteca|             1917550|
|      Olympiastadion|              815300|
|Maracan� - Est�di...|              813541|
|     Wembley Stadium|              778034|
| Estadio do Maracana|              741971|
+--------------------+--------------------+
only showing top 5 rows



In [17]:
#query 8 Matches held on 8th June 1958
matches_df.select('Home_Team_Name').filter(matches_df["Date_Time"].contains("08 Jun 1958")).count()

8

In [18]:
#query 9 max all type of goals scored year wise
goals = matches_df.withColumn('total_goals',  matches_df["Home_Team_Goals"] + matches_df["Opponent_Team_Goals"] + matches_df["Half-time_Home_Goals"] + matches_df["Half-time_Opponent_Goals"]).select('total_goals','year')
goals.groupBy('Year').max('total_goals').show()

+----+----------------+
|Year|max(total_goals)|
+----+----------------+
|1930|              13|
|1934|              11|
|1938|              12|
|1950|              12|
|1954|              21|
|1958|              14|
|1962|              12|
|1966|              13|
|1970|              10|
|1974|              15|
|1978|              10|
|1982|              14|
|1986|              10|
|1990|               8|
|1994|              10|
|1998|               9|
|2002|              12|
|2006|               9|
|2010|               8|
|2014|              13|
+----+----------------+
only showing top 20 rows



In [19]:
#query 10 describe type of goals which gives count,mean,min and ma
matches_df.describe(['Home_Team_Goals','Opponent_Team_Goals','Half-time_Home_Goals','Half-time_Opponent_Goals']).show()

+-------+------------------+-------------------+--------------------+------------------------+
|summary|   Home_Team_Goals|Opponent_Team_Goals|Half-time_Home_Goals|Half-time_Opponent_Goals|
+-------+------------------+-------------------+--------------------+------------------------+
|  count|               852|                852|                 852|                     852|
|   mean|1.8110328638497653|  1.022300469483568|  0.7089201877934272|      0.4284037558685446|
| stddev|1.6102551385229658| 1.0875733783096067|   0.937414128662808|      0.6912518906955025|
|    min|                 0|                  0|                   0|                       0|
|    max|                10|                  7|                   6|                       5|
+-------+------------------+-------------------+--------------------+------------------------+



In [20]:
#query 11 number of matches played in the year
matches_df.filter(matches_df.Year.like("1954")).count()

26

In [37]:
rdd =spark.sparkContext.textFile(r'C:\Users\jagad\Desktop\WorldCups.csv')

rdd.count()

21

In [38]:
# rddheader = rdd.first()
# header = spark.sparkContext.parallelize([rddheader])
# finalrdd = rdd.subtract(header)
# finalrdd.take(2)
header=rdd.first()
final_rdd = rdd.filter(lambda x: x!=header)
rdd = rdd.map(lambda line: (line.split(",")))

In [35]:
finalrdd.count()

20

In [200]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType, DoubleType

schema = StructType([StructField('Year', StringType(), True),
                         StructField('Country', StringType(), True),
                         StructField('Winner', StringType(), True),
                         StructField('Runners-Up', StringType(), True),
                         StructField('Third', StringType(), True),
                         StructField('Fourth', StringType(), True),
                         StructField('GoalsScored', StringType(), True),
                         StructField('QualifiedTeams', StringType(), True),
                         StructField('MatchesPlayed', StringType(), True),
                         StructField('Attendance', StringType(), True)])

df=spark.createDataFrame(rdd,schema=schema)

df = df.withColumn('GoalsScored', df['GoalsScored'].cast(IntegerType()))
df = df.withColumn('MatchesPlayed', df['MatchesPlayed'].cast(IntegerType()))
df = df.withColumn('QualifiedTeams', df['QualifiedTeams'].cast(IntegerType()))

In [86]:
#query 1,total number of goals scored by a winning team in all the matches
#rdd
final_rdd.filter(lambda x:x.split(','))\
.map(lambda x:(x.split(',')[2],x.split(',')[6]))\
.reduceByKey(lambda x,y:int(x)+int(y))\
.sortByKey().collect()


[('Argentina', 234),
 ('Brazil', 612),
 ('England', '89'),
 ('France', '171'),
 ('Germany', '171'),
 ('Germany FR', 352),
 ('Italy', 447),
 ('Spain', '145'),
 ('Uruguay', 158)]

In [91]:
#data frame
df.select('Winner','GoalsScored').groupBy('Winner').sum().sort('Winner').show()

+----------+----------------+
|    Winner|sum(GoalsScored)|
+----------+----------------+
| Argentina|             234|
|    Brazil|             612|
|   England|              89|
|    France|             171|
|   Germany|             171|
|Germany FR|             352|
|     Italy|             447|
|     Spain|             145|
|   Uruguay|             158|
|    Winner|            null|
+----------+----------------+



In [133]:
#query 2 countries with number of times won cup
final_rdd.filter(lambda x:x.split(',')[2]!='null').map(lambda x:(x.split(',')[2],1))\
.reduceByKey(lambda a,b:a+b)\
.sortByKey()\
.collect()

[('Argentina', 2),
 ('Brazil', 5),
 ('England', 1),
 ('France', 1),
 ('Germany', 1),
 ('Germany FR', 3),
 ('Italy', 4),
 ('Spain', 1),
 ('Uruguay', 2)]

In [132]:
#dataframe
df.select('Winner').groupBy('Winner').count().orderBy('Winner').show()


+----------+-----+
|    Winner|count|
+----------+-----+
| Argentina|    2|
|    Brazil|    5|
|   England|    1|
|    France|    1|
|   Germany|    1|
|Germany FR|    3|
|     Italy|    4|
|     Spain|    1|
|   Uruguay|    2|
|    Winner|    1|
+----------+-----+



In [153]:
# df = df.withColumn('Attendance', df['Attendance'].cast(LongType()))
#query 3, country where highest number of matches played in all the years 
final_rdd.filter(lambda x:x.split(',')[1]!='null').map(lambda x:(x.split(',')[1],x.split(',')[8]))\
.reduceByKey(lambda a,b:int(a)+int(b))\
.sortByKey().collect()

[('Argentina', '38'),
 ('Brazil', 86),
 ('Chile', '32'),
 ('England', '32'),
 ('France', 82),
 ('Germany', 102),
 ('Italy', 69),
 ('Korea/Japan', '64'),
 ('Mexico', 84),
 ('South Africa', '64'),
 ('Spain', '52'),
 ('Sweden', '35'),
 ('Switzerland', '26'),
 ('USA', '52'),
 ('Uruguay', '18')]

In [186]:

df.select('Country','MatchesPlayed').groupBy('Country').sum().sort('Country').show()

+------------+------------------+
|     Country|sum(MatchesPlayed)|
+------------+------------------+
|   Argentina|                38|
|      Brazil|                86|
|       Chile|                32|
|     Country|              null|
|     England|                32|
|      France|                82|
|     Germany|               102|
|       Italy|                69|
| Korea/Japan|                64|
|      Mexico|                84|
|South Africa|                64|
|       Spain|                52|
|      Sweden|                35|
| Switzerland|                26|
|         USA|                52|
|     Uruguay|                18|
+------------+------------------+



In [187]:
#query 4, records with country and winner same
final_rdd.map(lambda x:x.split(','))\
.filter(lambda x :x[1]==x[2]).collect()

[['1930',
  'Uruguay',
  'Uruguay',
  'Argentina',
  'USA',
  'Yugoslavia',
  '70',
  '13',
  '18',
  '590.549'],
 ['1934',
  'Italy',
  'Italy',
  'Czechoslovakia',
  'Germany',
  'Austria',
  '70',
  '16',
  '17',
  '363.000'],
 ['1966',
  'England',
  'England',
  'Germany FR',
  'Portugal',
  'Soviet Union',
  '89',
  '16',
  '32',
  '1.563.135'],
 ['1978',
  'Argentina',
  'Argentina',
  'Netherlands',
  'Brazil',
  'Italy',
  '102',
  '16',
  '38',
  '1.545.791'],
 ['1998',
  'France',
  'France',
  'Brazil',
  'Croatia',
  'Netherlands',
  '171',
  '32',
  '64',
  '2.785.100']]

In [188]:
#dataframe
df.filter(df.Country==df.Winner)\
.show()

+----+---------+---------+--------------+--------+------------+-----------+--------------+-------------+----------+
|Year|  Country|   Winner|    Runners-Up|   Third|      Fourth|GoalsScored|QualifiedTeams|MatchesPlayed|Attendance|
+----+---------+---------+--------------+--------+------------+-----------+--------------+-------------+----------+
|1930|  Uruguay|  Uruguay|     Argentina|     USA|  Yugoslavia|         70|            13|           18|       590|
|1934|    Italy|    Italy|Czechoslovakia| Germany|     Austria|         70|            16|           17|       363|
|1966|  England|  England|    Germany FR|Portugal|Soviet Union|         89|            16|           32|      null|
|1978|Argentina|Argentina|   Netherlands|  Brazil|       Italy|        102|            16|           38|      null|
|1998|   France|   France|        Brazil| Croatia| Netherlands|        171|            32|           64|      null|
+----+---------+---------+--------------+--------+------------+---------

In [195]:
#qurry 5 distinct countries
final_rdd.filter(lambda x:x.split(',')[1]!='null')\
.map(lambda x:x.split(',')[1]).distinct()\
.collect()


['Uruguay',
 'France',
 'Brazil',
 'Switzerland',
 'Mexico',
 'Argentina',
 'USA',
 'South Africa',
 'Italy',
 'Sweden',
 'Chile',
 'England',
 'Germany',
 'Spain',
 'Korea/Japan']

In [196]:
#dataframe
df.select(df.Country).distinct().show()

+------------+
|     Country|
+------------+
|      Sweden|
|     Germany|
|      France|
|   Argentina|
| Korea/Japan|
|       Chile|
|       Italy|
|       Spain|
|         USA|
|     Uruguay|
|      Mexico|
| Switzerland|
|      Brazil|
|     England|
|     Country|
|South Africa|
+------------+



In [197]:
#query 6 fetch record where country is USA
final_rdd.filter(lambda x:x.split(',')[1]=='Argentina')\
.collect()

['1994,USA,Brazil,Italy,Sweden,Bulgaria,141,24,52,3.587.538']

In [201]:
df.filter(df.Country=='USA').show()

+----+-------+------+----------+------+--------+-----------+--------------+-------------+----------+
|Year|Country|Winner|Runners-Up| Third|  Fourth|GoalsScored|QualifiedTeams|MatchesPlayed|Attendance|
+----+-------+------+----------+------+--------+-----------+--------------+-------------+----------+
|1994|    USA|Brazil|     Italy|Sweden|Bulgaria|        141|            24|           52| 3.587.538|
+----+-------+------+----------+------+--------+-----------+--------------+-------------+----------+

